# Sensorfusion - Autonomes Fahren


In [1]:
# Imports
import pandas as pd
import numpy as np
from pyds import MassFunction
import time as time
import math
from collections import deque

SciPy not found: some features will not work.


In [2]:
# Define dataframes from csvs
data1=pd.read_csv("DPE16_1.csv", sep=';', decimal=",")
data2=pd.read_csv("DPE16_2.csv", sep=';', decimal=",")
data3=pd.read_csv("DPE16_3.csv", sep=';', decimal=",")
data4=pd.read_csv("DPE16_4.csv", sep=';', decimal=",")

# Notizen
Jede Zeile entspricht Messung.
Verhältnis Breite/Höhe durch BBox-Werte.
Geschwindigkeit nur über Zeit (min Anzahl an Messwerten?) durch Abstand Differenz - Eigengeschwindigkeit
Beschleunigung über Zeit (sobald 2 Werte für Geschwindigkeit vorhanden: GeschwindigkeitNeu - GeschwindigkeitAlt

Konfidenz höher, je kleiner der Abstand
Omega: 1 - Konfidenz
omega: P L M F



In [3]:
#Initialize arrays to handle old values that are needed for calculations
speedArray = deque([None, None, None, None, None])
avgSpeedArray = deque([None, None, None, None, None])
accArray = deque([None, None, None, None, None])
omega = 'pmlf'

In [4]:
# Calculate bounding ratios from bounding box
# Higher value means bycicle or motorbike (1.6 - 2.4)
# lower value means car (0.4 - 1.0)
# not in the task but we think equal ratios should increase the probability for truck
# values around 1 mean truck (0.8 - 1.6)
def calcBoundingRatio(height, width):
    ratio = height / width
    
    return ratio
    
# -----------------------------------------------------------
# Manage saved values of speed and acceleration
# Only used internally
def saveSpeed(speed):
    global speedArray
    speedArray.append(speed)
    speedArray.rotate(-1)
    speedArray.pop()

def saveAvgSpeed(speed):
    global avgSpeedArray
    avgSpeedArray.append(speed)
    avgSpeedArray.rotate(-1)
    avgSpeedArray.pop()
    
def saveAcc(acc):
    global accArray
    accArray.append(acc)
    accArray.rotate(-1)
    accArray.pop()
# -----------------------------------------------------------

# Calculate speed
# Sensors are not accurate enough. They can only measure in 0,5m steps
# In an effort to fix this we handle an array containing the last 6 measurements for speed and accuracy
def calcSpeed(oldOwnSpeed, newOwnSpeed, oldDistance, newDistance):
    global speedArray
    # Check if there are null values for distance
    if (pd.isna(newDistance)):
        print('New Distance is NaN')
        print('Do not evaluate speed due to measurement error')
        saveSpeed(None)
        
        return
    
    # Calculate new speed and handle calculation
    difDistance = newDistance - oldDistance
    # We use average speed as ownSpeed because old and new speed can be slightly different
    ownSpeed = (oldOwnSpeed + newOwnSpeed) / 2
    speed = difDistance / 0.1 + ownSpeed
    saveSpeed(speed)
    
    # Calculate an average using the last 6 values to return
    speedArrayNotNone = [i for i in speedArray if i is not None]
    avgSpeed = sum(map(float,speedArrayNotNone)) / len(speedArrayNotNone)
    saveAvgSpeed(avgSpeed)
    
    return avgSpeed
    
# calculate acceleration with formula: dif speed / dif time
def calcAcceleration():
    global avgSpeedArray
    global accArray

    # Filter None values to avoid errors
    speedArrayNotNone = [i for i in avgSpeedArray if i is not None]
    arrayLength = len(speedArrayNotNone)
    # If not enough speed values are saved acceleration 0 is returned which will not influence the evaluation
    if (arrayLength < 2):
        return 0
    # Calculate new acceleration based on the last two speed values
    acc = (speedArrayNotNone[arrayLength - 1] - speedArrayNotNone[arrayLength - 2]) / 0.1
    saveAcc(acc)
    # Calculate average acceleration to minimize measurement errors
    # Even when taking averages the values differ too much to get realistic values
    accArrayNotNone = [i for i in accArray if i is not None]
    avgAcc = sum(map(float, accArrayNotNone)) / len(accArrayNotNone)

    return avgAcc


In [5]:
# Handle weighting

# Sensors have more accuracy on low distances
# This function changes omega according to the distance
def calcOmegaFromDistance(distance):
    omeg = 0
    if (distance < 20):
        omeg = 0.2
    elif (distance > 60):
        omeg = 0.5
    else:
        omeg = 0.2 + (0.0075 * distance - 0.15)
    return omeg
    
# Weight bounding box
# Higher value means bycicle or motorbike (1.6 - 2.4)
# lower value means car (0.4 - 1.0)
# not in the task but we think equal ratios should increase the probability for truck
# values around 1 mean truck (0.8 - 1.6)
    
# To optimise results omeg should be changed based on the values of bounding box...
# instead of only categorizing the calculations it would be better to scale the value
# To keep the script simple and development time low we did not implement this
def probabilityFromRatio(ratio, distance):
    omeg = calcOmegaFromDistance(distance)
    p = l = m = f = 0

    # Set weights according to ratio categories
    if ((ratio >= 1.6) & (ratio < 2.4)):
        m = f = (1 - omeg) / 2
    
    if ((ratio < 1.6) & (ratio >= 1.0)):
        l = (1 - omeg) / 2
    
    if ((ratio < 1.0) & (ratio >= 0.8)):
        p = l = (1 - omeg) / 2
    
    if ((ratio < 0.8) & (ratio >= 0.4)):
        p = (1 - omeg) / 2
     
    mass = MassFunction({'p': p, 'l': l, 'm': m, 'f': f, omega: omeg})
    
    return mass

# Weight Speed:
# PKW, LKW, Motorrad: 50km/h - 90km/h ==> ~13.89m/s - 25m/s
def probabilityFromSpeed(speed, distance):
    omeg = calcOmegaFromDistance(distance)

    # Task requirement
    
    p = l = m = f = 0

    if ((speed > 13.88) & (speed < 25)):
        m = p = l = (1 - omeg) / 3

    mass = MassFunction({'p': p, 'l': l, 'm': m, 'f': f, omega: omeg})
    
    return mass

# Weight acceleration
def probabilityFromAcc(acc, distance):
    # Increase omega because acceleration values differ a lot to make sure
    # measurement errors do not lead to wrong decisions
    omeg = calcOmegaFromDistance(distance) + 0.3

    p = l = m = f = 0

    # Task requirement
    # We set the value for acceleration to 4 which
    # we think might match a motorcyles acceleration but is higher than most cars (up to 3)
    # Reference: https://unfallanalyse.hamburg/index.php/ifu-lexikon/fahrmanoever/beschleunigung/#:~:text=Anfahrbeschleunigung%20Pkw%3A%201%2C5%20m,2%20bis%205%20m%2Fs%C2%B2
    if (acc > 4):
        m = (1 - omeg)

    mass = MassFunction({'p': p, 'l': l, 'm': m, 'f': f, omega: omeg})
    
    return mass

In [6]:
def iterateMeasurement(data, debug):
    
    massResult = None

    #for row in data.items():
    for index, row in data.iterrows():

        # Evaluate Ratio
        ratio = calcBoundingRatio(row['Bbox_Hoehe'], row['Bbox_Breite'])
        
        if(index==0):
            continue
            
        speed = calcSpeed(data.loc[(index - 1),'Eigengeschwindigkeit(m/s)'],
                            row['Eigengeschwindigkeit(m/s)'],
                            data.loc[(index - 1),'Abstand(m)'],
                            row['Abstand(m)'])
        
        acc = calcAcceleration()
        if(debug):
            # Data Print
            # If debugging requested: get output of calculated values foreach iteration
            print('---------------------------------------------------')
            print('Zeitpunkt:', row['t'])        
            print('Ration', ratio)  
            print('Speed: ', speed)
            print('Acceleration: ', acc)
        
        # Simulate 100 ms clock
        # Comment out for faster execution
        # time.sleep(0.1)

        # MassFunction 
        massR = probabilityFromRatio(ratio, row['Abstand(m)'])
        massS = probabilityFromSpeed(speed, row['Abstand(m)'])
        massA = probabilityFromAcc(acc, row['Abstand(m)'])
        mass = massR.combine_conjunctive(massS).combine_conjunctive(massA)
        if (massResult is None):
            massResult = mass
        else:
            massResult = massResult.combine_conjunctive(mass)
            
        if(debug):  
            # If debugging requested: get the output of weights for every iteration            
            print('MassR: ', massR)             
            print('MassS: ', massS)             
            print('MassA: ', massA)            
            print('Mass: ', mass)
            print('MassResult: ', massResult)
            
    # Print out after finishing the data
    print('MassResult: ', massResult)
    print('---------------------------------------------------')
    print('Wahrscheinlichkeiten für einzelne Fahrzeuge:')
    print('Pkw: ', massResult.pl({'p'}))
    print('Belief: ', massResult.bel({'p'}))
    print('Lkw: ', massResult.pl({'l'}))
    print('Belief: ', massResult.bel({'l'}))
    print('Motorrad: ', massResult.pl({'m'}))
    print('Belief: ', massResult.bel({'m'}))
    print('Fahrrad: ', massResult.pl({'f'}))
    print('Belief: ', massResult.bel({'f'}))
    print('=======================================================================')



# Execute the script for all dataframes we were given
# Please make sure the table head does not contain special chars such as ö
    # because we did not rename the columns but instead edited the csv files
iterateMeasurement(data1, False)
iterateMeasurement(data2, False)
iterateMeasurement(data3, False)
iterateMeasurement(data4, False)

NameError: name 'omega' is not defined